## TODO: ontdubbelen klopt niet. Beide waarden gaan eruit

Temp tabel met dubbele waarden maken. Dan alle rijen verwijderen waarvoor datum voorkomt in temp tabel
Daarna de unieke waarden in temp tabel toevoegen

In [2]:
import requests
import pandas as pd
import sqlalchemy
import psycopg2
import datetime

# connection variables
URL = 'football-fortunes.cj5o1qrvsyia.eu-west-1.rds.amazonaws.com'
database_name = 'postgres'
port = 5432
username = 'master'
ww = 'wachtwoord'

# Create SQL Alchemy database connection
db_url = f"postgresql://{username}:{ww}@{URL}:{port}/{database_name}"
engine = sqlalchemy.create_engine(db_url)
sqal_connection = engine.connect()

# Create Psychop connection
psy_connection = psycopg2.connect(
    host = URL,
    port = port,
    database = database_name,
    user= username,
    password = ww
)
psy_cursor = psy_connection.cursor()

# Haal de laatste datum op waarop dwa.d_datum is geupdate en stel vast wat delta is tov vandaag
# nieuwe update als update_datum minimaal 30 dagen geleden is
update_datum = pd.read_sql("SELECT update_datum FROM dsa.update WHERE tabelnaam = 'd_datum'", sqal_connection).at[0,'update_datum']
if (datetime.date.today() - update_datum).days >30:

    # Maak een tabel met alle data_waarden vanaf 2009 tm 150 dagen in de toekomst
    te_genereren_records = (datetime.date.today() - datetime.datetime(2009,1,1).date()).days + 150
    datumlijst = []
    for i in range(te_genereren_records):
        datum = datetime.datetime(2009,1,1).date() + datetime.timedelta(days=i)
        jaar = datum.year
        maand = datum.month
        if maand > 6:
            seizoen = f'{jaar}/{jaar+1}'
        else: 
            seizoen = f'{jaar-1}/{jaar}'
        if maand > 6:
            winterstop = 'voor'
        else:
            winterstop = 'na'
        datum_waarde = (datum, jaar, maand, seizoen, winterstop) # 
        datumlijst.append(datum_waarde)

    # TRUNCATE TABEL
    psy_cursor.execute("""
                        TRUNCATE dwa.d_datum;
                        """)

    # INSERT alle dataregels in de tabel dwa.d_datum en haal eventuele dubbele waarden eruit
    aantaldata = 0
    for datum in datumlijst:
        psy_cursor.execute("""
                            INSERT INTO dwa.d_datum 
                            VALUES (%s, %s, %s, %s, %s);                        
                            """
                            ,(datum[0], datum[1], datum[2], datum[3], datum[4]))
        aantaldata += 1

    psy_cursor.execute("""
                        UPDATE dsa.update
                        SET update_datum = current_date
                        WHERE tabelnaam = 'd_datum';                      
                        """)
                        
# Geef terug hoeveel nieuwe dataregel zijn toegevoegd
    print (f'aantal nieuwe data: {aantaldata}')   

# Commit de transactie en sluit de verbinding
sqal_connection.commit()
psy_connection.commit()
psy_cursor.close()
sqal_connection.close()
psy_connection.close()